In [ ]:
import pandas as pd
import numpy as np

shark_df = pd.read_excel('./shark-dataset.xls')
# create dataframe copy
original_df = shark_df.copy()
shark_df

In [ ]:
#sex wrangling with Tung's years

shark_df = dft.copy()

shark_df['sex'] = shark_df['sex'].str.strip()

# Replace specific values
shark_df['sex'] = shark_df['sex'].replace({
    'M': 'M', 
    'F': 'F',  
    'N': np.nan,  
    'M x 2': 'M', 
    'lli': np.nan,  
    '.': np.nan,  
    ' M': 'M'  
})

shark_df['sex']= shark_df['sex'].fillna('unknown')

#Calculate the counts of "M" and "F"
total_known = shark_df['sex'].value_counts()
m_count = total_known.get('M', 0)
f_count = total_known.get('F', 0)
total = m_count + f_count

#Calculate the percentages of "M" and "F"
if total > 0:
    m_percentage = m_count / total
    f_percentage = f_count / total
else:
    m_percentage = 0.5  # Default to equal distribution if no known values
    f_percentage = 0.5

# Determine the number of "Unknown" values
unknown_count = shark_df['sex'].value_counts().get('unknown', 0)

# Calculate how many "Unknown" values to fill with "M" and "F"
m_fill_count = int(m_percentage * unknown_count)
f_fill_count = unknown_count - m_fill_count  # Ensure all "Unknown" are assigned

# Get indices of the "Unknown" entries
unknown_indices = shark_df[shark_df['sex'] == 'unknown'].index

# Randomly shuffle the "Unknown" indices
shuffled_indices = np.random.permutation(unknown_indices)

# Split the shuffled indices into two groups for "M" and "F"
m_indices = shuffled_indices[:m_fill_count]
f_indices = shuffled_indices[m_fill_count:]

# Assign "M" and "F" to the split indices
shark_df.loc[m_indices, 'sex'] = 'M'
shark_df.loc[f_indices, 'sex'] = 'F'

# Verify replacements by checking updated counts
print(shark_df['sex'].value_counts())

In [ ]:
gender_counts = shark_df['sex'].value_counts()
print(gender_counts)

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(x='sex', data=shark_df, order=gender_counts.index)
plt.title('Shark Attacks by Gender')
plt.xlabel('Gender')
plt.ylabel('Number of Attacks')
plt.show()

In [ ]:
#age wrangling with Tung's years
def convert_descriptive_age(value):
    if pd.isnull(value):
        return np.nan
    value = str(value).strip().lower()
    if value in ["teen", "teens"]:
        return 15  # Approximate age for teenagers
    elif value == "adult":
        return 30  # General average for adult age
    elif value in ["middle age", '"middle-age"']:
        return 45  # Approximate age for middle age
    elif value == "elderly":
        return 70  # Approximate age for elderly
    elif value in ["a minor", "young"]:
        return 10  # Assume a minor is around 10 years old
    elif value == "infant" or value == "9 months" or value == "2 to 3 months":
        return 1  # Age 1 for infants
    elif "month" in value:
        return 1  # Treat other month values as infants
    return value

shark_df['age'] = shark_df['age'].apply(convert_descriptive_age)

def convert_to_first_age(value):
    if isinstance(value, str):
        numbers = re.findall(r'\d+', value)
        if numbers:
            return int(numbers[0])  
    return value

shark_df['age'] = shark_df['age'].apply(convert_to_first_age)

def convert_half_age(value):
    if isinstance(value, str) and "½" in value:
        # Replace "½" with ".5" and convert to float
        return float(value.replace("½", ".5"))
    return value  

shark_df['age'] = shark_df['age'].apply(convert_half_age)


#Convert any remaining irregular entries to NaN
def convert_irregular_entries(value):
    if isinstance(value, str) and not any(char.isdigit() for char in value):
        return np.nan  
    return value

shark_df['age'] = shark_df['age'].apply(convert_irregular_entries)

#convert to numeric
shark_df['age'] = pd.to_numeric(shark_df['age'], errors='coerce')

#Replace NaN values with the mode of the age column
age_mode = shark_df['age'].mode()[0]
shark_df['age'] = shark_df['age'].fillna(age_mode)

#convert type to int
shark_df['age'] = shark_df['age'].astype(int)

In [ ]:
age_bins = list(range(0, 101, 10))  # creates [0, 10, 20, ..., 100]
age_labels = [f"{age_bins[i]}-{age_bins[i+1] - 1}" for i in range(len(age_bins) - 1)]

# Apply pd.cut with the new bins and labels
shark_df['age_group'] = pd.cut(shark_df['age'], bins=age_bins, labels=age_labels, right=False)

# Calculate and display age group counts
age_group_counts = shark_df['age_group'].value_counts().sort_index()
print(age_group_counts)

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='age_group', data=shark_df, order=age_group_counts.index)
plt.title('Shark Attacks by Age Group')
plt.xlabel('Age Group')
plt.ylabel('Number of Attacks')
plt.show()